# Notebook que faz merge entre features issues,cs e email.

In [38]:
import pandas as pd
import numpy as np


dataset='eclipse'
# 1. Ler o arquivo CSV contendo o dataset
df_input1 = pd.read_csv(f'./3_output_merge/{dataset}/datasetClass_with_foc.csv')
df_input2 = pd.read_csv(f'./3_output_merge/{dataset}/datasetMethod_with_foc.csv')

# 2. Abrir o outro arquivo CSV com as colunas 'commit', 'feature1' e 'feature2'
df_pmd_cs = pd.read_csv(f'./4_output_pmd/{dataset}_cs_pmd.csv')
df_pmd_issues = pd.read_csv(f'./4_output_pmd/{dataset}_issues_pmd.csv')

# 3. Unir os dois dataframes com base na coluna 'commit'
# df_final = pd.merge(df1, df2, on='commit', how='left')

# 4. Salvar o dataframe resultante como um novo arquivo CSV
# df_final.to_csv('new_arquivo_entrada.csv', index=False)

In [39]:
df_input1.shape, df_input2.shape, df_pmd_cs.shape, df_pmd_issues.shape

((44605, 60), (360918, 42), (2370, 3), (2370, 3))

In [40]:
#adicionando as colunas cs e issues NO datasetClass

merge1 = pd.merge(df_input1, df_pmd_cs, on='commit', how='left')
merge1 = pd.merge(merge1, df_pmd_issues, on='commit', how='left')
merge1['cs'] = np.where(merge1['refactored'] == 1, merge1['cs_after'], merge1['cs_before'])
merge1['issues'] = np.where(merge1['refactored'] == 1, merge1['issues_after'], merge1['issues_before'])

merge1.shape

(44605, 66)

In [41]:
#adicionando as colunas cs e issues NO datasetMethod

merge2 = pd.merge(df_input2, df_pmd_cs, on='commit', how='left')
merge2 = pd.merge(merge2, df_pmd_issues, on='commit', how='left')
merge2['cs'] = np.where(merge2['refactored'] == 1, merge2['cs_after'], merge2['cs_before'])
merge2['issues'] = np.where(merge2['refactored'] == 1, merge2['issues_after'], merge2['issues_before'])

merge2.shape


(360918, 48)

In [42]:
import json


with open(f'./2_output_apiGithub/metrics2-{dataset}.json', 'r') as file:
    data = json.load(file)

# Now you can work with the data from the JSON file
# For example, you can access specific values using keys
devEmails = []

for project in data:
    if (len(project['metrics']['commitsRefactoringSha']) == 0):
        continue
    for sha in project['metrics']['commitsRefactoringSha']:
        myobj = {}
        # myobj['project'] = project['project_name']
        myobj['commit'] = sha
        devEmails.append(myobj)

for commit in devEmails:
    for project in data:
        if commit['commit'] in project['metrics']['commitsRefactoringSha']:
            commit['emails'] = project['metrics']['commitsRefactoringAutor']
    
for commit in devEmails:
    commit['emails'] = set(commit['emails'])
    

len(devEmails)

8014

In [43]:
devEmails_df = pd.DataFrame(devEmails)
merge1 = pd.merge(merge1, devEmails_df, on='commit', how='left')
merge2 = pd.merge(merge2, devEmails_df, on='commit', how='left')


In [44]:
merge1.shape, merge2.shape

((45291, 67), (372101, 49))

In [45]:
import os

directory = f'./5_output_merge2/{dataset}'
if not os.path.exists(directory):
    os.makedirs(directory)
    
merge1.to_csv(f'./5_output_merge2/{dataset}/datasetClass_with_foc_pmd_email.csv', index=False)
merge2.to_csv(f'./5_output_merge2/{dataset}/datasetMethod_with_foc_pmd_email.csv', index=False)

In [46]:
merge2.shape

(372101, 49)